In [2]:
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from corner import corner
from acor import acor
from utils.UL_uncert import UL_uncert

from astropy.time import Time
from matplotlib.colors import LogNorm

%matplotlib inline

In [3]:
plt.rcParams.update(plt.rcParamsDefault)
params = {'backend': 'pdf',
        'axes.labelsize': 12,
        'lines.markersize': 4,
        'font.size': 12,
        'xtick.major.size':6,
        'xtick.minor.size':3,
        'ytick.major.size':6,
        'ytick.minor.size':3,
        'xtick.major.width':0.7,
        'ytick.major.width':0.7,
        'xtick.minor.width':0.7,
        'ytick.minor.width':0.7,
        'lines.markeredgewidth':1.2,
        'axes.linewidth':1.125,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'savefig.dpi':200,
        'path.simplify':True,
        #'font.family':'serif',
        #'font.serif':'Computer Modern Roman',
        'mathtext.fontset':'cm',
        #'text.latex.preamble':'\\usepackage{amsmath}',
        'text.usetex':True,
         }

plt.rcParams.update(params)

# 11yr BWM plots

scripts to reproduce the figures in the 11yr BWM paper.

This is super messy, because the storage formatting evolved over many reruns...

## pulsar term Bayes factor

In [ ]:
with open('/home/pbaker/nanograv/bwm/sngl/detect_DE436/sngl_BF.json') as fin:
    bf_dict = json.load(fin)

which PSRs have $\mathcal{B}>1$ ?

In [ ]:
[(k,v) for k,v in bf_dict.items() if v>1]

### sngl BF hist

In [ ]:
ar_kwargs = {'arrowstyle':'->'}

bfs = [x for x in bf_dict.values()]
plt.figure(figsize=(5,3))

plt.hist(bfs, bins=20);
plt.xlabel('Bayes factor')
plt.ylabel('Counts')
plt.axvline(x=1, ls=':', color='C1')

psrs = ['J1909$-$3744', 'J1747$-$4036', 'J1744$-$1134']
arrow_loc = [(1.32, 1.1), (1.63, 1.1), (3.50, 1.1)]
text_loc = [(1.1, 7), (1.6, 4), (2.8, 4)]

for p, xy, xytext in zip(psrs, arrow_loc, text_loc):
    plt.annotate(p, xy, xytext, arrowprops=ar_kwargs)

plt.tight_layout()
plt.savefig('/home/pbaker/psrTerm_BF.pdf')

use for logBF
```
ar_kwargs = {'arrowstyle':'->'}

bfs = [x for x in bf_dict.values()]
plt.figure(figsize=(5,3))

plt.hist(np.log10(bfs), bins=25);
plt.xlabel('bayes factor')
plt.ylabel('counts')
plt.axvline(x=0, ls=':', color='C1')

psrs = ['J1909-3744', 'J1747-4036', 'J1744-1134']
arrow_loc = [(0.14, 1.1), (0.22, 1.1), (0.53, 1.1)]
text_loc = [(0.02, 4), (0.20, 3), (0.40, 2.2)]

for p, xy, xytext in zip(psrs, arrow_loc, text_loc):
    plt.annotate(p, xy, xytext, arrowprops=ar_kwargs)

plt.tight_layout()
plt.savefig('/home/pbaker/psrTerm_BF.pdf')
```

## UL v T

### load UL v. T data

11yr:

In [ ]:
# binned by source orientation reruns
with open('/home/pbaker/nanograv/bwm/ULvT/ULs.json', 'r') as fin:
    ULmarg = json.load(fin)

key = 'DE430'
ULs_430 = [x[1] for x in ULmarg[key]]
low_430 = [x[1]-x[2] for x in ULmarg[key]]
high_430 = [x[1]+x[2] for x in ULmarg[key]]

key = 'DE436'
ULs_436 = [x[1] for x in ULmarg[key]]
low_436 = [x[1]-x[2] for x in ULmarg[key]]
high_436 = [x[1]+x[2] for x in ULmarg[key]]

key = 'DE436_BE'
ULs_BE = [x[1] for x in ULmarg[key]]
low_BE = [x[1]-x[2] for x in ULmarg[key]]
high_BE = [x[1]+x[2] for x in ULmarg[key]]

T11 = [x[0] for x in ULmarg[key]]
T11 = Time(T11, format='mjd').decimalyear


# use BayesEphem for Rate ULs
key = 'DE436_BE'
UL_11 = np.array(ULmarg[key]).T

9yr:

In [ ]:
col_9yr = np.loadtxt('ULvT/9yr_cols.dat', dtype=str)
UL_9yr = np.loadtxt('ULvT/9yr_UL.dat')
dUL_9yr = np.loadtxt('ULvT/9yr_dUL.dat')
T9 = np.loadtxt('ULvT/9yr_Ts.dat')

# only show BayesEphem
ii = list(col_9yr).index('DE436_BE')
high_9 = UL_9yr[ii] + dUL_9yr[ii]
low_9 = UL_9yr[ii] - dUL_9yr[ii]
ULs_9 = UL_9yr[ii]

11yr pulsar term:

In [3]:
sngl_runs = '/home/pbaker/nanograv/bwm/sngl/uplim_DE436'

psrlist = '/home/pbaker/nanograv/data/nano11/11yr_34.txt'
with open(psrlist, 'r') as f:
    all_psrs = [line.strip() for line in f]
all_psrs.sort()

UL_dict = {}
for psr in all_psrs:
    ULfile = os.path.join(sngl_runs, psr, "ULvT_{:s}.dat".format(psr))
    UL_dict[psr] = np.loadtxt(ULfile)
    UL_dict[psr][0] = Time(UL_dict[psr][0], format='mjd').decimalyear

### make plots

In [ ]:
al = 0.25

plt.close('all')

fig = plt.figure(figsize=(5,4))
ax1, ax2 = fig.subplots(2, 1, sharex=True)

ax1.fill_between(T11, low_BE, high_BE, color='k', alpha=al)
ax1.plot(T11, ULs_BE, color='k', label='11 yr')

ax1.fill_between(T9, low_9, high_9, color='C2', alpha=al)
ax1.plot(T9, ULs_9, color='C2', label=' 9y r')

ax1.legend(loc='upper center', fontsize=12)
ax1.set_yscale('log')
ax1.set_ylabel(r'$95\%$ UL (strain)', fontsize=12)

ax2.fill_between(T11, low_436, high_436, color='C0', alpha=al)
ax2.plot(T11, ULs_436, color='C0', label='DE436')

ax2.fill_between(T11, low_430, high_430, color='C1', alpha=al)
ax2.plot(T11, ULs_430, color='C1', label='DE430')

ax2.fill_between(T11, low_BE, high_BE, color='k', alpha=al)
ax2.plot(T11, ULs_BE, color='k', label='BayesEphem')

ax2.legend(loc='upper center', fontsize=12)
ax2.set_yscale('log')
ax2.set_xlabel(r'Burst epoch', fontsize=12)
ax2.set_ylabel(r'$95\%$ UL (strain)', fontsize=12)
plt.tight_layout()

plt.savefig("/home/pbaker/UL_v_t.pdf")

In [ ]:
sngl_runs = '/home/pbaker/nanograv/bwm/sngl/uplim_DE436'
pname = 'J1713+0747'

ULfile = os.path.join(sngl_runs, pname, "ULvT_{:s}.dat".format(pname))
J1713_opt = np.loadtxt(ULfile)

J1713_Ts = Time(J1713_opt[0], format='mjd').decimalyear

J1713_ULs = [x[1] for x in J1713_opt.T]
J1713_low = [x[1]-x[2] for x in J1713_opt.T]
J1713_high = [x[1]+x[2] for x in J1713_opt.T]


# earth-term best source orientation for each time bin (from ULvt_post.ipynb)
best = np.array(
    [1.44490506e-13, 6.22070408e-14, 2.19012534e-14, 1.51206219e-14,
     1.28900134e-14, 6.88746799e-15, 7.46864473e-15, 7.78535327e-15,
     4.71788123e-15, 5.57351006e-15, 5.75463446e-15, 6.61798374e-15,
     5.38088219e-15, 5.22685056e-15, 5.15507171e-15, 5.10055013e-15,
     5.84093593e-15, 4.13696718e-15, 5.07611726e-15, 4.18810852e-15,
     4.16668178e-15, 4.22520830e-15, 3.68823550e-15, 3.79931560e-15,
     4.01828926e-15, 4.47221972e-15, 2.99191590e-15, 4.93018684e-15,
     3.48198989e-15, 4.19956015e-15, 2.53736492e-15, 4.12775499e-15,
     3.85474276e-15, 3.03049650e-15, 4.43941884e-15, 4.21373834e-15,
     5.21945653e-15, 5.32721146e-15, 8.77647508e-15, 1.31618897e-14]
)

In [ ]:
plt.close()
fig = plt.figure(figsize=(5,3))

plt.fill_between(T11, low_BE, high_BE, color='k', alpha=al)
plt.plot(T11, ULs_BE, color='k', label='sky ave. Earth term')

plt.plot(T11, best, ls='--', color='k', label="best sky Earth term")

#plt.plot(*J1713_UL, color='k', label=pname+" sky-ave")
plt.fill_between(J1713_Ts, J1713_low, J1713_high, color='gray', alpha=al)
plt.plot(J1713_Ts, J1713_ULs, ls='--', color='gray', label=pname+" opt. source")


plt.legend(loc='upper center')
plt.yscale('log')
plt.xlabel(r'Burst epoch')
plt.ylabel(r'$95\%$ UL (strain)');

plt.tight_layout()
plt.savefig("/home/pbaker/UL_v_t_compare.pdf")

### rate ULs

In [ ]:
# 9yr 
dT_9 = np.mean(np.diff(T9))
hsort_9 = np.sort(UL_9yr[0])
Ns_9 = np.arange(len(hsort_9))+1

# 11yr
dT_11 = np.mean(np.diff(UL_11[0]))/365.25
hsort_11 = np.sort(UL_11[1])
Ns_11 = np.arange(len(hsort_11))+1

# 11yr PSR-term
#sngl_ULs = UL_11yr[0]  # uncomment for joint earth-term + psr-term
rescale = np.pi

sngl_ULs = []  # psr term only
for _,uls,_ in UL_dict.values():
    sngl_ULs = np.hstack([sngl_ULs, uls])

dT_psr = np.mean(np.diff(J1713_opt[0]))/365.25
hsort_psr = np.sort(sngl_ULs) * rescale
Ns_psr = np.arange(len(hsort_psr)) + 1

5yr ULs from Dusty:

In [ ]:
#NG5 ULs
ng5_e = np.loadtxt("/home/pbaker/ng5_eTerm.txt").T
ng5_p = np.loadtxt("/home/pbaker/ng5_pTerm.txt").T

Rate predictions from Islo, et al. (2019)

In [ ]:
dat = np.load("rate_vs_strain_Sesana_finegrain_10.npy").T

rate UL plot:

In [ ]:
wid=2
fig = plt.figure(figsize=(9,3))
ax1, ax2 = fig.subplots(1, 2)

# compare with 
ax1.plot(*dat, color='C3', lw=wid, label='predicted rate')
ax1.plot(hsort_psr, 1/(Ns_psr*dT_psr), color='C0', ls='--', lw=wid, label='11 yr pulsar term')
ax1.plot(hsort_11, 1/(Ns_11*dT_11), color='C0', lw=wid, label='11 yr Earth term')


# zoom
ax2.plot(hsort_psr, 1/(Ns_psr*dT_psr), color='C0', ls='--', lw=wid, label='11 yr pulsar term')
ax2.plot(hsort_11, 1/(Ns_11*dT_11), color='C0', lw=wid, label='11 yr Earth term')

ax2.plot(*ng5_p, color='C1', ls='--', lw=wid,label='5 yr pulsar term')
ax2.plot(*ng5_e, color='C1', lw=wid, label='5 yr Earth term')

for ax in fig.axes:
    ax.set_xlabel('Strain, $h_\mathrm{gw}$')#, fontsize=16)
    ax.set_ylabel('Event rate UL ($N$/yr)')#, fontsize=16)
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.legend()

ax1.set_xlim([1e-22, 1e-11])
ax2.set_xlim([1e-14, 1e-11])
    
plt.tight_layout()
plt.savefig('/home/pbaker/rateUL_compare.pdf')

In [ ]:
# median sky sensitivity compare:

thr = 2.5e-14
ii = np.searchsorted(ng5_e[0], thr)
ng5_thr = ng5_e[1][ii]

ii = np.searchsorted(hsort_11, thr)
ng11_thr = 1/(Ns_11*dT_11)[ii]

print(ng5_thr / ng11_thr)


ii = np.searchsorted(ng5_p[0], thr)
ng5_thr = ng5_p[1][ii]

ii = np.searchsorted(hsort_psr, thr)
ng11_thr = 1/(Ns_psr*dT_11)[ii]

print(ng5_thr / ng11_thr)

In [ ]:
# Large h compare (Earth Term)

thr = 1e-12
ii = np.searchsorted(ng5_e[0], thr)
ng5_thr = ng5_e[1][ii]

ii = np.searchsorted(hsort_11, thr)
ng11_thr = 1/(Ns_11*dT_11)[ii]

print(ng5_thr / ng11_thr)


In [ ]:
# Large h compare (pulsar-term)

thr = ng5_p[0][-1]
ng5_thr = ng5_p[1][-1]

ii = np.searchsorted(hsort_psr, thr)
ng11_thr = 1/(Ns_psr*dT_11)[ii]

print(ng5_thr / ng11_thr, thr)

## skymap

In [4]:
import healpy as hp
import skyplot_tools as spt
from matplotlib.colors import LogNorm

In [5]:
psr_dict = {}
with open('11yr_locs.txt', 'r') as f:
    for line in f:
        line = line.split()
        psr_dict[line[0]] = [float(line[1]), float(line[2])]
    
psr_locs = []
for p in all_psrs:
    psr_locs.append(psr_dict[p])
psr_locs = np.array(psr_locs)

In [6]:
nside = 8 #corresponds to 768 pixels
npix = hp.nside2npix(nside)

In [7]:
rundir = '/home/pbaker/nanograv/bwm/fixsky/latetime_BE/'
run, skymap, Ns, cors = np.loadtxt(rundir+'ULs.dat').T

print("min = {:.2e}; max = {:.2e}".format(min(10**skymap), max(10**skymap)))

min = 1.09e-14; max = 1.12e-13


In [8]:
zmin = 1e-14
zmax = 1.2e-13
skymap_exp = 10**skymap
norm=LogNorm(vmin=zmin, vmax=zmax)

plt.close('all')
plt.figure(0, figsize=(7, 5))
ax = plt.subplot(111, projection='astro mollweide')
ax.grid()
spt.outline_text(ax)

spt.healpix_heatmap(skymap_exp, cmap='viridis_r', norm=LogNorm(vmin=zmin, vmax=zmax))

ax.scatter(psr_locs[:,0], psr_locs[:,1], 12**2,
          marker='*', facecolor='w', edgecolors='k', alpha=0.9)

cbar = plt.colorbar(orientation='horizontal', pad=0.05)
plt.suptitle(r'$95\%$ Characteristic Strain Upper Limit, $h_{95}$', y=0.05)
plt.grid(linestyle='dotted', color='k')
plt.tight_layout()
plt.savefig('/home/pbaker/skymap_11yr.pdf')
plt.show()

KeyError: b'tcss3583'

In [ ]:
10**np.median(skymap)

## Dropout

In [4]:
with open('/home/pbaker/nanograv/bwm/dropout/dropout.json') as fin:
    drop_9 = json.load(fin)

with open('/home/pbaker/nanograv/bwm/dropout_11/dropout.json') as fin:
    drop_11 = json.load(fin)

In [5]:
psrkeys = np.array(list(drop_9.keys()))
Os_9 = np.array(list(drop_9.values()))
Os_11 = np.array([drop_11[key] for key in psrkeys])

In [6]:
psrnames = np.array([
    'B1855$+$09',
    'J0030$+$0451',
    'J0613$-$0200',
    'J1012$+$5307',
    'J1024$-$0719',
    'J1455$-$3330',
    'J1600$-$3053',
    'J1614$-$2230',
    'J1640$+$2224',
    'J1713$+$0747',
    'J1741$+$1351',
    'J1744$-$1134',
    'J1853$+$1303',
    'J1909$-$3744',
    'J1918$-$0642',
    'J2010$-$1323',
    'J2145$-$0750',
    'J2317$+$1439'
])

In [7]:
idx = np.argsort(Os_9)
xs = np.arange(len(Os_9))

fig = plt.figure(figsize=(5,3))
ax = fig.add_subplot(111)

ax.plot(xs, Os_9[idx[::-1]], linestyle='none', marker='o', color='C1', label='9 yr')
ax.plot(xs, Os_11[idx[::-1]], linestyle='none', marker='o', color='C0', label='11 yr')
ax.axhline(y=1, color='k', ls=':')
ax.set_xticks(xs)
ax.set_xticklabels(psrnames[idx[::-1]], rotation=80)
ax.set_ylim([1/100, 1000])
ax.set_yscale('log')
ax.set_ylabel('Bayes factor')
plt.yticks([0.01, 0.1, 1, 10, 100, 1000], ["0.01", "0.1", "1", "10", "100", "1000"])

ax.legend(loc=0)

plt.tight_layout()
plt.savefig("/home/pbaker/dropout.pdf")